# Hey, maikol here

In [1]:
%load_ext autoreload
%autoreload 2

### File functions

`check_dirs_existance`

In [2]:
from maikol_utils import check_dirs_existance

dirs = ["./data", "./config", "./logs"]
# Raises KeyError if any path is missing
try:
    check_dirs_existance(dirs)
except KeyError as e:
    print(e)  # e.g. "Some paths were not found: ['./config', './logs']"


"Some paths were not found: ['./data', './config', './logs']"


`make_dirs`

In [ ]:
from maikol_utils import make_dirs

dirs = ["./data/input", "./data/output"]
# Safely creates all these directories if they don't exist
make_dirs(dirs)

`save_json` & `load_json`

In [5]:
from maikol_utils import save_json, load_json

data = {"users": ["alice", "bob"], "count": 2}
# Saves to outputs/users.json, creating the folder if needed
save_json("outputs/users.json", data)

# Attempt to load; if not found, returns {}
settings = load_json("outputs/users.json")
print(settings)  # {"users": ["alice", "bob"], "count": 2}

Saving output at outputs/users.json...
Loading output from outputs/users.json...
{'users': ['alice', 'bob'], 'count': 2}


### Printing

`print_separator`

In [3]:
from maikol_utils import print_separator

print_separator("START PROCESS", sep_type="START")
print_separator("STEP COMPLETE", sep_type="SHORT")
print_separator("FINAL RESULTS", sep_type="SUPER")



                                                         START PROCESS                                                          


________________________________
         STEP COMPLETE          

                                                         FINAL RESULTS                                                          



`print_color` & `print_warn`

In [5]:
from maikol_utils import print_color, print_warn, print_error

print_color("All systems go", color="green")
print_color("Unexpected value", color="red")

# Prints ⚠️Missing config file!⚠️ in yellow
print_warn("Missing config file!")

# Prints ❌Missing config file!❌ in yellow
text = print_error("No model found!")

All systems go
Unexpected value
⚠️Missing config file!⚠️
❌No model found!❌


`print_status`

In [6]:
import time
from maikol_utils import print_status

for i in range(1, 4):
    print_status(f"Processing page {i}/3...")
    time.sleep(0.5)
# At the end, call print_status("") to clear the line
print_status("")


`clear_bash` & `print_clear_bash`
- this wont work here :)

In [10]:
import time
from maikol_utils import clear_bash, print_clear_bash


print("Line A")
print("Line B")
clear_bash(2)   # removes both lines from the terminal
print("Clean slate!")


print("Loading data...")
time.sleep(1)
print_clear_bash("✔️ Data loaded successfully!", n_lines=1)



Line A
Line B
Clean slate!
Loading data...
✔️ Data loaded successfully!
